In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn

In [ ]:
df = pd.read_csv('../input/bluebook-for-bulldozers/TrainAndValid.csv',low_memory=False)
df.head()

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df.columns

In [ ]:
fig,ax = plt.subplots()
ax.scatter(df['saledate'][:1000],df['SalePrice'][:1000])

In [ ]:
df['SalePrice'].plot.hist()

In [ ]:
df.saledate.dtype

### Parsing Dates

when we work with the time series data, we want to enrich the time & date component as much as possible.

we can do that by telling pandas which of our columns has dates in it using `parse_dates` parameter.

In [ ]:
# Import data again but this time parse dates
df = pd.read_csv('../input/bluebook-for-bulldozers/TrainAndValid.csv',low_memory=False,
                parse_dates=['saledate'])
df.saledate.dtype


In [ ]:
df['saledate'][:5]

In [ ]:
fig,ax = plt.subplots()
ax.scatter(df['saledate'][:1000],df['SalePrice'][:1000])

In [ ]:
df.head()

In [ ]:
df.head().T

### Sort DataFrame by saledate
When working with time series data, it's a good idea to sort it by date.

In [ ]:
# Sort dataframe by date order.
df.sort_values(by=['saledate'],inplace=True,ascending=True)
df.saledate.head()

In [ ]:
# Make a copy of the original DataFrame.
df_tmp = df.copy()


### Add datetime parameters for saledate column

In [ ]:
df_tmp['saleYear'] = df_tmp.saledate.dt.year
df_tmp['saleMonth'] = df_tmp.saledate.dt.month
df_tmp['saleDay'] = df_tmp.saledate.dt.day
df_tmp['saleDayOfWeek'] = df_tmp.saledate.dt.dayofweek
df_tmp['saleDayOfYear'] = df_tmp.saledate.dt.dayofyear

In [ ]:
df_tmp.head().T

In [ ]:
#now we can remove saledate
df_tmp.drop('saledate',axis=1,inplace=True)

In [ ]:
df_tmp.state.value_counts()

## Modelling

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_jobs=-1,
                             random_state=42)
model.fit(df_tmp.drop('SalePrice',axis=1),df_tmp['SalePrice'])

### Convert String to categories

One way we can turn all of our data into numbers is by converting them into pandas categories.

In [ ]:
pd.api.types.is_string_dtype(df_tmp['UsageBand'])

In [ ]:
# Find the columns which contains strings
for label, content in df_tmp.items():
    if pd.api.types.is_string_dtype(content):
        print(label)

In [ ]:
df_tmp.info()

In [ ]:
for label, content in df_tmp.items():
    if pd.api.types.is_string_dtype(content):
        df_tmp[label] = content.astype('category').cat.as_ordered()

In [ ]:
df_tmp.info()

In [ ]:
df_tmp.state.cat.categories

In [ ]:
df_tmp.state.cat.codes

In [ ]:
# check missing data
df_tmp.isnull().sum()/len(df_tmp)

### Fill missing values

### Fill numeric columns first

In [ ]:
for label,content in df_tmp.items():
    if pd.api.types.is_numeric_dtype(content):
        print(label)

In [ ]:
# Check for which nuemeric columns have null values:
for label,content in df_tmp.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label)

In [ ]:
# Fill missing rows with median
for label,content in df_tmp.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            df_tmp[label+'_is_missing'] = pd.isnull(content)
            df_tmp[label] = content.fillna(content.median())
            

In [ ]:
df_tmp.isna().sum()

# Filling and turning categorical variables into numbers

In [ ]:
# Check for columns which aren't numeric:
for label,content in df_tmp.items():
    if not pd.api.types.is_numeric_dtype(content):
        print(label)

In [ ]:
pd.Categorical(df_tmp['state']).codes

In [ ]:
# Turn categorical variables into numbers and fill missing
for label,content in df_tmp.items():
    if not pd.api.types.is_numeric_dtype(content):
        df_tmp[label+'_is_missing'] = pd.isnull(content)
        df_tmp[label] = pd.Categorical(content).codes+1

In [ ]:
df_tmp.isnull().sum()

In [ ]:
df_tmp.info()

In [ ]:
model = RandomForestRegressor(n_jobs=-1,
                             random_state=42)
model.fit(df_tmp.drop('SalePrice',axis=1),df_tmp['SalePrice'])

In [ ]:
model.score(df_tmp.drop('SalePrice',axis=1),df_tmp['SalePrice'])

In [ ]:
df_tmp.saleYear.value_counts()

In [ ]:
# Splitting data into training and validation sets
df_val = df_tmp[df_tmp.saleYear== 2012]
df_train = df_tmp[df_tmp.saleYear != 2012]
len(df_val), len(df_train)

In [ ]:
# Split into X and y
X_train,y_train = df_train.drop('SalePrice',axis=1),df_train['SalePrice']
X_valid,y_valid = df_val.drop('SalePrice',axis=1),df_val['SalePrice']

### Building an evaluation Function

In [ ]:
# Creating a RMSLE
from sklearn.metrics import mean_squared_log_error,mean_absolute_error,r2_score
def rmsle(y_test,y_preds):
    
    return np.sqrt(mean_squared_log_error(y_test,y_preds))
def show_scores(model):
    train_preds = model.predict(X_train)
    val_preds = model.predict(X_valid)
    scores = {'Training MAE':mean_absolute_error(y_train,train_preds),
             'Valid MAE':mean_absolute_error(y_valid,val_preds),
             'Traing RMSLE':rmsle(y_train,train_preds),
             'valid Rmsle':rmsle(y_valid,val_preds),
             'Training R^2': r2_score(y_train,train_preds),
             'valid R^2':r2_score(y_valid,val_preds)}
    return scores

# Testing model on a subset( to Tune Hyperparameters)

In [ ]:
%%time
model = RandomForestRegressor(n_jobs=-1,
                             random_state=42)
model.fit(X_train,y_train)

In [ ]:
model = RandomForestRegressor(n_jobs=-1,
                             random_state=42,
                              max_samples=10000,)

In [ ]:
%%time

model.fit(X_train,y_train)

In [ ]:
show_scores(model)

### Hyperparameter Tuning with RandomizedSearchCV

In [ ]:
%%time
from sklearn.model_selection import RandomizedSearchCV

rf_grid ={'n_estimators':np.arange(10,100,10),
         'max_depth':[None,3,5,10],
         'min_samples_split':np.arange(2,20,2),
         'min_samples_leaf':np.arange(1,20,2),
         'max_features':[0.5,1,'sqrt','auto'],
         'max_samples':[10000]}
rs_model = RandomizedSearchCV(RandomForestRegressor(n_jobs=-1,
                                                   random_state=42,),
                                                   param_distributions=rf_grid,
                                                   n_iter=5,
                                                   cv=5,
                                                   verbose=True)
rs_model.fit(X_train,y_train)

In [ ]:
rs_model.best_params_

In [ ]:
show_scores(rs_model)

In [ ]:
%%time
ideal_model = RandomForestRegressor(n_estimators=90,
                                   min_samples_split= 2,
                                    min_samples_leaf= 13,
                                   max_samples= 10000,
                                   max_features= 'sqrt',
                                   max_depth= None,
                                   n_jobs=-1)
ideal_model.fit(X_train,y_train)

In [ ]:
show_scores(ideal_model)

# Make predictions on Test Data

In [ ]:
df_test = pd.read_csv('../input/bluebook-for-bulldozers/Test.csv',
                     low_memory=False,
                     parse_dates=['saledate'])
df_test.head()

In [ ]:
test_preds = ideal_model.predict(df_test)

In [ ]:
df_test.isnull().sum()

In [ ]:
df_test.info()

### preprocessing the data (getting the test dataset as training data)

In [ ]:
def preprocess_data(df):
    """
    Perform transformations on df and return transformed df.
    
    """
    # parse date
    df['saleYear'] = df.saledate.dt.year
    df['saleMonth'] = df.saledate.dt.month
    df['saleDay'] = df.saledate.dt.day
    df['saleDayOfWeek'] = df.saledate.dt.dayofweek
    df['saleDayOfYear'] = df.saledate.dt.dayofyear
    
    df.drop('saledate',axis=1,inplace=True)
    
    # Fill the numeric rows with median
    for label,content in df.items():
        if pd.api.types.is_numeric_dtype(content):
            if pd.isnull(content).sum():
                df[label+'_is_missing']=pd.isnull(content)
                df[label] = content.fillna(content.median())
                
    # Fill categorical missing data and turn categories into numbers
    for label,content in df.items():
        if not pd.api.types.is_numeric_dtype(content):
            df[label+'_is_missing']=pd.isnull(content)
            #we add +1 to category code
            df[label] = pd.Categorical(content).codes+1
            
    
    return df

In [ ]:
# process test data
df_test = preprocess_data(df_test)
df_test.head()

In [ ]:
df_train.head()

In [ ]:
# Make predictions on updated test data
test_preds = ideal_model.predict(df_test)

In [ ]:
# We can find how the columns differ using sets
set(X_train.columns)-set(df_test.columns)

In [ ]:
# Manually adjust df_test to have auctioneerID_is_missing column
df_test['auctioneerID_is_missing']=False
df_test

In [ ]:
test_preds = ideal_model.predict(df_test)

In [ ]:
# Format the predictions into the same format Kaggle is after:
df_preds = pd.DataFrame()
df_preds['SalesID'] =df_test['SalesID']
df_preds['SalesPrice'] = test_preds
df_preds
